**Montar colab con drive**

In [ ]:
#para montar mi drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

**Librerias**

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

In [ ]:
import os
import cv2

#carpetaOrigen = '/content/gdrive/MyDrive/validacion/x_validacionImg/'

def cargarSetImagenes(carpetaOrigen = ''):
  contenido = os.listdir(carpetaOrigen)
  imagenes = []
  for fichero in contenido:
      img = cv2.imread(carpetaOrigen + fichero,cv2.IMREAD_GRAYSCALE)
      imagenes.append(img)
  return np.array(imagenes)



In [ ]:
train_data = cargarSetImagenes("/content/gdrive/MyDrive/IA-SARS/archivosBinarioSeparado/entrenamiento/x_entrenamientoImg/")
train_dataY = cargarSetImagenes("/content/gdrive/MyDrive/IA-SARS/archivosBinarioSeparado/entrenamiento/y_entrenamientoImg/")
test_data = cargarSetImagenes("/content/gdrive/MyDrive/IA-SARS/archivosBinarioSeparado/validacion/x_validacionImg/")
test_dataY = cargarSetImagenes("/content/gdrive/MyDrive/IA-SARS/archivosBinarioSeparado/validacion/y_validacionImg/")

In [ ]:
print(train_data)
print(len(train_dataY))
print(len(test_data))
print(len(test_dataY))

In [ ]:
print(test_data[0].shape) 

**Normalizacion de datos**

In [ ]:
###Etapa de preprocesamiento de datos
def preprocesamiento(array): # mandar a llamar mis imagenes de entrenamiento x y y
# Normalizamos (valores entres 0 y 1) las imagenes y los casteamos al formato correcto, escala de grises. 
    arrayNormalizado = []
    for matrizImg in array:
      arrayImg = matrizImg.astype("float32") / 255.0
      #print("Se hizo la division");
      #print("Size: "+str(len(arrayImg))+'x'+str(len(arrayImg[0])) )
      arrayImg = np.reshape(arrayImg[:,:], (172, 172, 1))#Imágenes de 172x172px con un solo canal(escala de grises)
      arrayNormalizado.append(arrayImg)
    return np.array(arrayNormalizado)#Retornamos el arreglo de datos 

***Cargar los datos entrenamiento y validacion ***

In [ ]:
#Cargamos los datos, en este caos como usaremos el dataset para codificar y decodificar, de imagen a imgagen, no es necesario cargar las etiquetas
###(train_data, _), (test_data, _) = mnist.load_data()#* cargar x entrenamiento  validacion x 

# Normalizamos y ajustamos la imagenes al formato correcto**}----
train_data = preprocesamiento(train_data)#Prepocesamos el conjunto de entrenamiento
train_dataY = preprocesamiento(train_dataY)
test_data = preprocesamiento(test_data)#Prepocesamos el conjunto de pruebas
test_dataY = preprocesamiento(test_dataY)

print(train_data.shape)


**creacion del modelo**

In [ ]:
##Vamos a crear el modelo agregando elementos al objeto layers
##La primera configuración es el tamaño de la capa de entrada, es el tamaño de las imágenes (28x28x1)
input = layers.Input(shape=(172, 172, 1))

# Encoder(Codificador)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder(Decodificador)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)#La funcion Conv2DTranspose se encarga de aumentar el tamaño de la matriz de salida
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()#resume la arquitectura

**Entrenamiento**

In [ ]:
#Vamos a entrenar el modelo usando como entrada imagenes con ruido, esperando que a la salida nos entregue una imagen limpia.
#En este caso estamos usando dos conjuntos de imagenes, las que tienen ruido y las que estan limpias(las esperadas)

autoencoder.fit(
    x= train_data,#Conjunto de entrenamiento con ruido
    y=train_dataY, #Conjunto de entrenamiento sin ruido (estas son las que esperamos que entregue el modelo)
    epochs=100,
    batch_size=128, 
    shuffle = True,
    verbose=1,
    validation_data=(test_data, test_dataY),#La validación se hará con el conjunto con ruido comparado con las esperadas
)

**Pruebas**

In [ ]:
#Primero vamos a probar el modelo con las imágenes de prueba sin agregar ruido para ver como la recupera
predictions = autoencoder.predict(test_data)
#display(predictions, test_data )#Imprimimos el resultado de algunas imagenes usadas con su predicción (arriba=imagen de entrada, abajo=imagen resultado del modelo)
#Ahora el modelo es capaz de limpiar las imágenes, obteniendo una imagen diferente a la de entrada

In [ ]:
#Agregadas mi
from numpy import save
save('/content/gdrive/MyDrive/IA-SARS/resultados_entrenamiento/predictions20.npy', predictions)
#Siempre llegar hasta aqui

In [ ]:
#Agregadas mi
# load numpy array from npy file
from numpy import load
# load array
predictions = load('/content/gdrive/MyDrive/IA-SARS/resultados_entrenamiento/predictions20.npy')


In [ ]:
test_dataNN = predictions.astype("float32") * 255.0 # normalizamos la matriz de predicciones


In [ ]:
from scipy.spatial import distance # importo el objeto que me permitara usar la funcion de distancia

test_dataNN = predictions.astype("float32") * 255.0 # normalizamos la matriz de predicciones
#print(test_dataNN[0])

def fun(value):#Definimos la funcion que se aplicara a cada elemento de la matris
    distances = []#Se dedine el arreglo donde se guardaran las distancias
    distances.append(distance.euclidean((value,0),(0,0)))#0 Se calcula la distancia del valor respecto a 0
    distances.append(distance.euclidean((value,0),(64,0)))#1 Se calcula la distancia del valor respecto a 64
    distances.append(distance.euclidean((value,0),(128,0)))#2 Se calcula la distancia del valor respecto a 128
    distances.append(distance.euclidean((value,0),(255,0)))#3 Se calcula la distancia del valor respecto a 255
    distanceMin = min(distances) # Se obtiene la minima distancia del arreglo de distancias
    indexMin = distances.index(distanceMin) #En base a la minima distancia se obtiene el indice correspondiente para saber a cual esta mas cerca
    valueParse = 0 #Se define la variable donde se retornara el valor que corresponde a la distancia mas corta euclaniana
    if indexMin == 0: # Si el indice es 0 se entiende que esta mas cerca a 0
      valueParse = 0
    elif indexMin == 1:# Si el indice es 1 se entiende que esta mas cerca a 64
      valueParse = 64
    elif indexMin == 2: # Si el indice es 2 se entiende que esta mas cerca a 128
      valueParse = 128
    elif indexMin == 3:# Si el indice es 3 se entiende que esta mas cerca a 255
      valueParse = 255

    return valueParse # Se retorna el valor a la aproximacion correspondiente

vfunc = np.vectorize(fun)#  coonvierte la funcion a una F. Numpy
result = vfunc(test_dataNN) #Se implementa la funcion

#print(test_dataNN)

print(result)#Se implimen los resultados

In [ ]:
test_dataNN = test_data.astype("float32") * 255.0

def compareSecuenses(secuencePrediction,secuenceTest):
  coinciden = 0
  secPre = np.asarray(secuencePrediction).reshape(-1)
  secTes = np.asarray(secuenceTest).reshape(-1)
  for index in range(0,len(secPre)-181):
    #print(secPre[index],'-',secTes[index])
    if secPre[index] == secTes[index]:
      coinciden = coinciden + 1

  return coinciden


totalCoincidents = []
for index in range(0,len(test_dataNN)):
  totalCoincidents.append(compareSecuenses(result[index],test_dataNN[index]))
  
print('promedio: ',sum(totalCoincidents)/29403) 

In [ ]:
sum(totalCoincidents)/29403) 